In [ ]:
!pip install flask-ngrok

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Dataset/iris.csv')

dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
Y = dataset['Species'] # Target prediksi
x = dataset.drop(['Id', 'Species'], axis=1) # Data Id tidak diperlukan

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, Y, test_size=0.15, random_state=10)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(x)
# data_x = pd.DataFrame(X, columns=x.columns)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression())])
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [ ]:
y_pred = pipeline.predict(x_test)

In [ ]:
# logreg = LogisticRegression()
# logreg.fit(x_train, y_train)
# y_pred = logreg.predict(x_test)

In [ ]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(pipeline.score(x_test, y_test))) # .score() -> Accuracy
print(f1_score(y_test, y_pred, average='macro'))
print(precision_score(y_test, y_pred, average='macro'))
print(recall_score(y_test, y_pred, average='macro'))

Accuracy of logistic regression classifier on test set: 1.00
1.0
1.0
1.0


save model tarining

In [ ]:
import pickle

In [ ]:
filename = 'model.dd' # Nama yang digunakan bebas
pickle.dump(pipeline, open(filename, 'wb')) 

loading model

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
y_pred_loaded = loaded_model.predict(x_test)

In [ ]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(loaded_model.score(x_test, y_test))) # .score() -> Accuracy
print(f1_score(y_test, y_pred_loaded, average='macro'))
print(precision_score(y_test, y_pred_loaded, average='macro'))
print(recall_score(y_test, y_pred_loaded, average='macro'))

Accuracy of logistic regression classifier on test set: 1.00
1.0
1.0
1.0


In [ ]:
x_test

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
87,6.3,2.3,4.4,1.3
111,6.4,2.7,5.3,1.9
10,5.4,3.7,1.5,0.2
91,6.1,3.0,4.6,1.4
49,5.0,3.3,1.4,0.2
60,5.0,2.0,3.5,1.0
72,6.3,2.5,4.9,1.5
67,5.8,2.7,4.1,1.0
39,5.1,3.4,1.5,0.2
55,5.7,2.8,4.5,1.3


In [ ]:
#Contoh prediksi

# Membuat sebuah list dengan dimensi yang sama

#test_data = [[0.553333, -1.744778, 0.364699, 0.133226]] # (index 87 / index 0 pada x_test

# for index in x_test:
#   prediction_example = loaded_model.predict(x_test)

# print(prediction_example)

# Membuat sebuah list dengan dimensi yang sama

test_data = [[5.1, 3.5, 1.4, 0.2]] # Data sebelum di pre-process

prediction_example = loaded_model.predict(test_data)

prediction_example[0]


'Iris-setosa'

flask ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask

In [ ]:
app = Flask(__name__) # Membuat sebuah object Flask

@app.route("/home")
def home():
    return "<h1>Hello World!</h1>"

In [ ]:
from flask import jsonify, request

In [ ]:
@app.route('/classify', methods=['POST'])
def classification():
  sepal_width = float(request.json['sepal_width'])
  sepal_length = float(request.json['sepal_length'])
  petal_length = float(request.json['petal_length'])
  petal_width = float(request.json['petal_width'])

  array = [[sepal_width, sepal_length, petal_length, petal_width]]
  prediction_example = loaded_model.predict(array)
  output_class = (prediction_example[0])

  # if sepal_width < 0.5 and petal_length > 0.6:
  #   output_class = "Iris-Versicolor"
  # else:
  #   output_class = "Iris-Virginica"

  return jsonify({
      "output": output_class
  })

In [ ]:
run_with_ngrok(app)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://68ee08379e3d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Oct/2020 05:13:23] "POST /classify HTTP/1.1" 200 -
